# 可迭代的对象、迭代器、生成器

## 迭代
- 目的：惰性获取数据
- 所有生成器都是迭代器，迭代器用于从集合中去除元素，而生成器用于凭空生成元素
- 生成器可以在无穷集合中生成

In [1]:
import re
import reprlib
from random import randint

# 第一版sentence类
- 通过将句子划分成序列，实现迭代

In [2]:
RE_WORD = re.compile('\w+')

In [3]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self, index):
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

## iter
- 解释器需要迭代对象x时，会自动调用iter(x)
- 检查顺序：对象是否实现了``__iter__``，实现了就获取迭代器->是否实现``__getitem__``方法，是吸纳了就创建迭代器->抛出typeerror异常

## 可迭代对象、迭代器
- 两者不同，python从可迭代的对象中获取迭代器，可迭代的对象是吸纳了``__iter__``方法返回迭代器
- for语句隐含实现了获取迭代器，再进行迭代
- for、列表推导、元组拆包都自动捕获StopIteration，保证正常迭代直至结束
- 迭代器的定义：**实现了无参数的``__next__``方法，返回序列中的下一个元素；如果没有元素了，那么抛出StopIteration异常。Python中的迭代器还实现了``__iter__``方法，因此迭代器也可以迭代**
- 可迭代的对象包含``__iter__``方法，每次都实例化返回一个新的迭代器；迭代器要实现``__next__``方法，返回单个元素，此外还要实现``__iter__``方法，返回迭代器本身。即：迭代器可以迭代，但是可迭代的对象不是迭代器。
- 一般不建议让一个类既是可迭代的对象，又是迭代器，因为我们需要能从同一个可迭代的实例中获取多个独立的迭代器，而且各个迭代器要能维护自身的内部状态。要实现这样就需要将迭代的实现放在可迭代对象的``__iter__``方法中，每次实例化一个新的迭代器，而不是在可迭代对象中直接实现``__next__``方法

## 实现标准的迭代器接口
- 包含``__next__``和``__iter__``两个接口
- 两个接口再collections.abc.Iterator中定义
    - Iterator类定义了``__next__``抽象方法
    - Iterator类继承自Iterable类
    - Iterable类定义了``__iter__``抽象方法
    - ``Iterable.__iter__``方法返回一个Iterator实例
    - 具体的Iterator类必须实现``__next__``方法
    - ``Iterator.__iter__``方法直接返回实例本身

## 第二版Sentence类
- 这一版Sentence类定义了``__iter__``方法，返回一个迭代器
- 因此定义了一个迭代器类，该类实现了``__next__``方法，同时也满足迭代器的定义，实现了``__iter__``方法并返回本身

In [4]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return SentenceIterator(self.words)


class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0

    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

    def __iter__(self):
        return self

## 生成器函数
- 包含yield关键字的函数就是生成器函数
- 调用生成器函数会返回一个生成器对象
- 生成器函数会创建一个生成器对象，包装生成器函数的定义体
- 把生成器传给next函数时，生成器函数会向前执行到函数体的下一个yield语句，返回yeild后面的产出值
- 生成器不会以常规的方式返回值，而是产出值
- 生成器函数中的return语句会抛出StopIteration异常

## 第三版Sentence类
- 通过生成器函数实现

In [5]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:
            yield word
        return

## 第四版Sentence类
- 惰性实现
- 只在需要的实现生成单词，而不是一次性生成整个单词列表
- findall可以改为惰性实现finditer

In [6]:
class Sentence:
    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 第五版Sentence类
- 生成器表达式可以理解为列表推导的惰性版本
- 生成器表达式是语法糖，完全可以替换成生成器函数

In [7]:
class Sentence:
    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## itertools以及内置中包含了许多实现优秀的生成器函数

In [8]:
import itertools
help(itertools)

Help on built-in module itertools:

NAME
    itertools - Functional tools for creating and using iterators.

DESCRIPTION
    Infinite iterators:
    count(start=0, step=1) --> start, start+step, start+2*step, ...
    cycle(p) --> p0, p1, ... plast, p0, p1, ...
    repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times
    
    Iterators terminating on the shortest input sequence:
    accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
    chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ... 
    chain.from_iterable([p, q, ...]) --> p0, p1, ... plast, q0, q1, ... 
    compress(data, selectors) --> (d[0] if s[0]), (d[1] if s[1]), ...
    dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails
    groupby(iterable[, keyfunc]) --> sub-iterators grouped by value of keyfunc(v)
    filterfalse(pred, seq) --> elements of seq where pred(elem) is False
    islice(seq, [start,] stop [, step]) --> elements from
           seq[start:stop:step]
    starmap(fun, seq) --> fun(*seq

## yield from
- 使用yield from实现：生成器函数需要产出另一个生成器生成的值

In [9]:
def chain(*iterables):
    for i in iterables:
        yield from i

In [10]:
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

## iter函数的另一种用法
- 传入两个参数，第一个参数是一个可调用的对象，用于不断调用产生值；第二个参数是哨符
- 当产生的值返回这个哨值时，触发迭代器抛出StopIteration异常

In [11]:
def d6():
    return randint(1, 6)


d6_iter = iter(d6, 1)
d6_iter

In [12]:
for roll in d6_iter:
    print(roll)

3
4


## 作者提到的一个实例
- 需求：读取两种不同格式的文件，转为json写入文件
- 问题：存在两种不同的读文件格式，可能需要交叉读不同类型的文件；文件太大，不能一次性读入内存
- 解决：解耦读写逻辑，将两种类型的文件读操作用两个生成器实现，写文件时根据类型调用不同的生成器，获取不同的文件并转格式写入